In [1]:
import pandas as pd
import numpy as np
import sys
import os
import gensim
import spacy
import altair as alt
from hyphen import Hyphenator
from gensim.parsing.preprocessing import preprocess_string, STOPWORDS 
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_numeric
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from sklearn.metrics import accuracy_score
import textstat
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn import metrics 
nltk.download('averaged_perceptron_tagger')
cwd = os.getcwd()
from sklearn.svm import SVC
from operator import itemgetter
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
import operator

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\afhar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:

df_tokenized = pd.read_pickle('df_tokenized')
features = df_tokenized.original_text
labels = df_tokenized.label

In [ ]:
#pipeline parameters
parameters = \
    [ \
#        {
#           'clf': [MultinomialNB()],
#            'tf-idf__stop_words': ['english'],
#            'tf-idf__lowercase' : ['True'],
#            'tf-idf__max_df' : [1.0,0.9,0.7],
#            'tf-idf__min_df' : [1,0.01, 0.1, 0.25],
#            'tf-idf__ngram_range' : [(1,1), (1,2), (1,3)],
#            'clf__alpha': [0.001, 0.1, 1, 10, 100]
#        },

        {
           'clf': [SVC()],
            'tf-idf__stop_words': ['english'],
            'tf-idf__lowercase' : ['True'],
            'tf-idf__ngram_range' : [(1,2),(1,3)],
            'clf__C': [0.001, 0.1, 1, 10, 100],
            'clf__kernel': ['poly', 'rbf'],
        },

#        {
#            'clf': [DecisionTreeClassifier()],
#            'tf-idf__stop_words': ['english'],
#            'tf-idf__lowercase' : ['True'],
#            'tf-idf__max_df' : [1.0,0.9,0.7],
#            'tf-idf__min_df' : [1,0.01, 0.1, 0.25],
#            'tf-idf__ngram_range' :[(1,1), (1,2), (2,3)],
#            'clf__criterion': ['gini','entropy'],
#            'clf__splitter': ['best','random'],
#            'clf__class_weight':['balanced', None]
#        }
#        {
#           'clf': [KNeighborsClassifier()],
#            'clf__algorithm':['auto'],
#            'clf__n_neighbors': [5,25, 50],
#            'clf__p': [1,2],
#            'clf__weights': ('uniform', 'distance'),
#            'clf__metric': ('minkowski', 'chebyshev'),
#            'tf-idf__stop_words': ['english'],
#            'tf-idf__lowercase' : ['True'],
#            'tf-idf__ngram_range' : [(1,2)]
            
#        },
#         {
#             'clf': [SGDClassifier()],
#             'clf__loss':['hinge'],
#             'clf__penalty': ['l2','l1'],
#             'clf__alpha': [0.0001,0.001,0.1],
#             'clf__warm_start': ['False'],
#             'tf-idf__stop_words': ['english'],
#             'tf-idf__lowercase' : ['True'],
#             'tf-idf__ngram_range' : [(1,2),(1,3)]
             
#         },
    ]

#evaluating multiple classifiers
#based on pipeline parameters
#-------------------------------
result=[]

for params in parameters:

    #classifier
    clf = params['clf'][0]

    #getting arguments by
    #popping out classifier
    params.pop('clf')

    #pipeline
    steps = [('tf-idf', TfidfVectorizer()), ('clf',clf)]

    #cross validation using
    #Grid Search
    grid = GridSearchCV(Pipeline(steps), param_grid=params, cv=3,n_jobs = -1,verbose=10)
    grid.fit(features, labels)

    #storing result
    result.append\
    (
        {
            'grid': grid,
            'classifier': grid.best_estimator_,
            'best score': grid.best_score_,
            'best params': grid.best_params_,
            'cv': grid.cv
        }
    )

#sorting result by best score
result = sorted(result, key=operator.itemgetter('best score'),reverse=True)

#saving best classifier
grid = result[0]['grid']


Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


In [ ]:
import joblib
joblib.dump(grid, 'classifier_bigrams_SVC.pickle')
result

In [ ]:
#Naive Bayes classifier-0.6936880967904339, 0.72 on kaggle
#KNN classifier - 0.6347176407599342,
#SGD classifier- 0.6447663945363414,

In [ ]:
#import joblib
#joblib.dump(grid, 'classifier.pickle')
#result

In [25]:
#$tfidf = TfidfVectorizer(stop_words = 'english', lowercase = 'True', ngram_range = (1,3) )
#tf_f =tfidf.fit_transform(features)


array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [31]:
#make the predictions
submit_df = pd.read_pickle('test_data.pkl')
submit_df.head()
features = submit_df.original_text_x
y_pred = grid.predict(features)
df = pd.DataFrame()
df['id'] =submit_df.index
df['label'] = y_pred
df.to_csv('NB_tdidf_only.csv',index = False)

,id_x,original_text_x,label_x,tokens,pos,nouns,verbs,Pnouns,adjectives,adverbs,...,id_y,original_text_y,label_y,clean_text,word_count,clean_text_no_stop,word_count_no_stop,Nsyll,AoA_Kup_lem,Perc_known_lem
21108,21108,"Cobalt -LRB- II , III -RRB- oxide is chemical ...",NaN,"[Cobalt, LRB, II, III, RRB, oxide, is, chemica...","[(Cobalt, NNP), (LRB, NNP), (II, NNP), (III, N...",9,1,6,1,0,...,21108,"Cobalt -LRB- II , III -RRB- oxide is chemical ...",NaN,"[cobalt, lrb, ii, iii, rrb, oxide, is, chemica...",13,"[cobalt, lrb, ii, iii, rrb, oxide, chemical, c...",10,2.400000,11.846000,1.000000
108997,108997,They then took a boat trip from Dunglass Burn ...,NaN,"[They, then, took, a, boat, trip, from, Dungla...","[(They, PRP), (then, RB), (took, VBD), (a, DT)...",24,5,13,4,2,...,108997,They then took a boat trip from Dunglass Burn ...,NaN,"[they, then, took, a, boat, trip, from, dungla...",56,"[took, boat, trip, dunglass, burn, east, along...",39,1.392857,5.601429,0.998214
72335,72335,"Outdoor public toilets -LRB- in the street , a...",NaN,"[Outdoor, public, toilets, LRB, in, the, stree...","[(Outdoor, NNP), (public, JJ), (toilets, NNS),...",9,1,3,1,0,...,72335,"Outdoor public toilets -LRB- in the street , a...",NaN,"[outdoor, public, toilets, lrb, in, the, stree...",17,"[outdoor, public, toilets, lrb, street, around...",12,1.666667,5.516667,1.000000
116453,116453,When Niklas Sundin and Johann Larsson left Ham...,NaN,"[When, Niklas, Sundin, and, Johann, Larsson, l...","[(When, WRB), (Niklas, NNP), (Sundin, NNP), (a...",22,2,16,2,0,...,116453,When Niklas Sundin and Johann Larsson left Ham...,NaN,"[when, niklas, sundin, and, johann, larsson, l...",34,"[niklas, sundin, johann, larsson, left, hammer...",26,1.636364,7.157273,0.995455
102736,102736,The standards development of HTTP has been coo...,NaN,"[The, standards, development, of, HTTP, has, b...","[(The, DT), (standards, NNS), (development, NN...",30,4,22,1,2,...,102736,The standards development of HTTP has been coo...,NaN,"[the, standards, development, of, http, has, b...",56,"[standards, development, http, coordinated, in...",37,2.400000,8.924500,0.975000
106402,106402,"There are 33 -LRB- some EMS text say 25 , coun...",NaN,"[There, are, 33, LRB, some, EMS, text, say, 25...","[(There, EX), (are, VBP), (33, CD), (LRB, NNP)...",10,3,3,4,0,...,106402,"There are 33 -LRB- some EMS text say 25 , coun...",NaN,"[there, are, 33, lrb, some, ems, text, say, 25...",26,"[33, lrb, ems, text, say, 25, counting, sacral...",19,1.500000,7.209286,0.956429
60892,60892,"Life Emerson was born in Boston , Massachusett...",NaN,"[Life, Emerson, was, born, in, Boston, Massach...","[(Life, NNP), (Emerson, NNP), (was, VBD), (bor...",4,2,4,0,0,...,60892,"Life Emerson was born in Boston , Massachusett...",NaN,"[life, emerson, was, born, in, boston, massach...",9,"[life, emerson, born, boston, massachusetts, 1...",6,1.000000,5.285000,1.000000
75481,75481,Professor Reginald Crundall Punnett FRS -LRB- ...,NaN,"[Professor, Reginald, Crundall, Punnett, FRS, ...","[(Professor, NNP), (Reginald, NNP), (Crundall,...",19,3,18,1,0,...,75481,Professor Reginald Crundall Punnett FRS -LRB- ...,NaN,"[professor, reginald, crundall, punnett, frs, ...",33,"[professor, reginald, crundall, punnett, frs, ...",26,3.200000,10.102000,0.978000
45263,45263,In education,NaN,"[In, education]","[(In, IN), (education, NN)]",1,0,0,0,0,...,45263,In education,NaN,"[in, education]",2,[education],1,4.000000,6.700000,1.000000
70412,70412,"On November 3 , 2004 , N. P. Johnsens Firework...",NaN,"[On, November, 3, 2004, N, P, Johnsens, Firewo...","[(On, IN), (November, NNP), (3, CD), (2004, CD...",6,1,6,0,0,...,70412,"On November 3 , 2004 , N. P. Johnsens Firework...",NaN,"[on, november, 3, 2004, n, p, johnsens, firewo...",11,"[november, 3, 2004, n, p, johnsens, fireworksf...",9,3.000000,6.335000,1.000000
